In [ ]:
import requests
def get_data():
    data = []
    timestamps = [1611488086,1614166486,1616585686,1619264086,1621856086,1624534486,1627126486,1629804886]
    for timestamp in timestamps:
        url = f'https://api.pushshift.io/reddit/search/submission/?subreddit=Anxiety&size=100&after={timestamp}'
        data.extend(requests.get(url).json()['data'])
    return data

In [ ]:
def dataset():
    for i in get_data():
        try:
            text = i['selftext']
            if(len(text) < 1000 and len(text) > 100):
                #not using any text too large or too small
                yield text
        except KeyError:
            #No text component in here
            pass
        except StopIteration:
            break

In [ ]:
import pandas as pd
data = []
def append_row(text,value):
    data.append([text,value])

def save_data():
    ds = pd.DataFrame(data = data,columns=['text','label'])
    ds.to_csv('reddit_anxiety.csv',index=False)

In [ ]:
from IPython.display import display
from ipywidgets import interactive_output, fixed
import ipywidgets as widgets

button = widgets.Button(description="Set Value")
ds = dataset()
slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.5,description="Anxiety",layout=widgets.Layout(width='50%'))
text_area = widgets.Textarea(
    layout = widgets.Layout(width='100%',height='400px'),
)
text = next(ds)
text_area.value = text

box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    width='80%'
)
data_index = widgets.Label('position:0')
def clicked(b):
    global text,ds,data_index
    anxiety_level = slider.value
    append_row(text,anxiety_level)
    try:
        text = next(ds)
    except StopIteration:
        global box
        save_data()
        box.close()
    text_area.value = text
    curr_idx = int(data_index.value[9:])
    data_index.value = f'position:{curr_idx + 1}'

box = widgets.VBox(
    [
        text_area,
        data_index,
        slider,
        button
    ],
    layout=box_layout
)
button.on_click(clicked)
display(box)

In [ ]:
save_data()

In [ ]:
valid = pd.DataFrame(data=list(ds),columns=['text'])

In [ ]:
valid.to_csv('valid.csv',index=False)

In [ ]:
train = pd.read_csv('reddit_anxiety.csv')

In [ ]:
train.head()

In [ ]:
valid.head()